In [511]:
import pandas as pd
import numpy as np
import polars as pl


In [512]:
folder = '.\\input_files\\'
file = '952294234_adventist-health---bakersfield_standardcharges.csv'

dtypes = {
    'Procedure Code': str, 
    'Revenue Code': str, 
    'Modifier1': str,
    'Modifier2': str,   
    'Rx Unit Multiplier': str 
}
df = pd.read_csv(folder + file, dtype=dtypes, skiprows=3)


C:\Users\adria\AppData\Local\Temp\ipykernel_7336\346380062.py:11: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(folder + file, dtype=dtypes, skiprows=3)


In [513]:
df.rename(columns={
    'Procedure Code': 'local_code',
    'Procedure Description': 'description',
    'Price Tier': 'setting',
    'Revenue Code': 'rev_code',
    'CPT HCPCS Code': 'code',
    'NDC Code': 'ndc',
    'Rx Unit Multiplier': 'drug_hcpcs_multiplier'
    
}, inplace=True)

In [514]:
df = df[~df['local_code'].isin(['Diagnosis Related Group Code', 'Shoppable Services Code'])]

In [515]:
cols = df.columns.tolist()
id_vars = cols[:9]
value_vars = cols[9:]


df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer', value_name='rate')

In [516]:
df['ndc'].fillna(pd.NA, inplace=True)

In [517]:
mapping = {
    'Gross Charge': 'gross',
    'Discounted Cash Price': 'cash',
    'De-identified minimum negotiated charge': 'min',
    'De-identified maximum negotiated charge': 'max'
}

df['payer_category'] = df['payer'].map(mapping).fillna('payer')

In [518]:
df['plan'] = ''
df['plan'] = np.where(df['payer'].str.contains('ALL_PAYER_-_ALL_OTHER_PLANS'), 'ALL PAYER - ALL OTHER PLANS',
                      np.where(df['payer'].str.contains('ALL_OTHER_PLANS'), 'ALL OTHER PLANS',
                               np.where(df['payer'].str.contains('ALL_PLANS'), 'ALL PLANS', df['plan'])))


df['payer'] = df['payer'].str.replace('ALL_PAYER_-_ALL_OTHER_PLANS|ALL_PLANS|ALL_OTHER_PLANS', '', regex=True).str.strip('-_')

In [519]:
df.loc[df['drug_hcpcs_multiplier'] == '0', 'drug_hcpcs_multiplier'] = pd.NA

In [520]:
df['modifiers'] = df['Modifier1'] + df['Modifier2']
df.drop(['Modifier1', 'Modifier2'], axis=1, inplace=True)

In [521]:
cols = ['setting', 'code', 'ndc', 'local_code', 'plan', 'modifiers']
df[cols] = df[cols].astype(str)
df.loc[:, cols].fillna('', inplace=True)

In [522]:
df['setting'].fillna('', inplace=True)

In [523]:
df['rate'] = df['rate'].astype(float)

In [524]:
df.loc[df['code'].str.match(r'^\d{3}$'), 'ms_drg'] = df['code']

In [527]:
df.loc[df['ndc'].isin(['<NA>', 'nan']), 'ndc'] = ''
df.loc[df['setting'].isin(['<NA>', 'nan']),'setting'] = ''
df.loc[df['modifiers'].isin(['<NA>', 'nan']),'modifiers'] = ''

df['ms_drg'].fillna('', inplace=True)
df['rev_code'].fillna('', inplace=True)

df['drug_hcpcs_multiplier'].fillna(pd.NA, inplace=True)

In [528]:
df

,local_code,description,setting,rev_code,code,ndc,drug_hcpcs_multiplier,payer,rate,payer_category,plan,modifiers,ms_drg
0,1010391_1,METANEPHRINES 1.2 17 171,Inpatient,0301,83835,,<NA>,Gross Charge,15.06,gross,,,
1,1010391_1,METANEPHRINES 1.2 17 171,Outpatient,0301,83835,,<NA>,Gross Charge,15.06,gross,,,
2,1010406_1,LEAD 1.1 17 170,Outpatient,0301,83655,,<NA>,Gross Charge,10.63,gross,,,
3,1010406_1,LEAD 1.1 17 170,Inpatient,0301,83655,,<NA>,Gross Charge,10.63,gross,,,
4,1022134_1,CREATININE BLOOD 1.2 17 173,Inpatient,0301,82565,,<NA>,Gross Charge,4.31,gross,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
761455,P9047,"INFUSION, ALBUMIN (HUMAN), 25%, 50 ML",,,P9047,,<NA>,WESTERN_GROWERS_ASSURANCE,584.55,payer,ALL PLANS,,
761456,S0119,"ONDANSETRON, ORAL, 4 MG (FOR CIRCUMSTANCES FAL...",,,S0119,,<NA>,WESTERN_GROWERS_ASSURANCE,128.88,payer,ALL PLANS,,
761457,U0003,INFECTIOUS AGENT DETECTION BY NUCLEIC ACID (DN...,,,U0003,,<NA>,WESTERN_GROWERS_ASSURANCE,63.39,payer,ALL PLANS,,
761458,U0005,INFECTIOUS AGENT DETECTION BY NUCLEIC ACID (DN...,,,U0005,,<NA>,WESTERN_GROWERS_ASSURANCE,34.50,payer,ALL PLANS,,
